In [ ]:
#p354 소득 예측 모델 만들기
import pandas as pd
df = pd.read_csv('adult.csv')
df.info()

#전처리
#타겟 변수 전처리
df['income'].value_counts(normalize = True)
import numpy as np
df['income'] = np.where(df['income'] == '>50K', 'high', 'low')
df['income'].value_counts(normalize = True)

#불필요한 변수 제거
df = df.drop(columns = 'fnlwgt')
#문자 타입 변수를 숫자 타입으로 변경
#원핫 인코딩
df_tmp = df[['sex']]
df_tmp.info()
df_tmp['sex'].value_counts()
#원핫 인코딩 적용
df_tmp = pd.get_dummies(df_tmp)
df_tmp.info() #bool type
df_tmp[['sex_Female', 'sex_Male']].head()

target = df['income'] #income 추출
df = df.drop(columns = 'income') #income 제거
df = pd.get_dummies(df) #문자 타입 변수 원핫 인코딩
df['income'] = target
df.info(max_cols = np.inf)

#데이터 분할
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df,
                                     test_size = 0.3, #테스트 세트 비율
                                     stratify = df['income'], #타겟 변수 비율 유지
                                     random_state = 1234) #난수 고정
#train
df_train.shape #(34189, 108)
df_train['income'].value_counts(normalize = True)
#test
df_test.shape #(14653, 108)
df_test['income'].value_counts(normalize = True)

#의사결정나무 모델 만들기
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state = 1234, #난수 고정
                                  max_depth = 3) #나무 깊이
train_x = df_train.drop(columns = 'income') #예측 변수 추출
train_y = df_train['income'] #타겟 변수 추출

model = clf.fit(X = train_x, y = train_y) #모델 만들기

#모델 구조 살펴보기
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi': '100', #해상도 설정
                     'figure.figsize': [12, 8]}) #그래프 크기 설정
tree.plot_tree(model,
               feature_names = list(train_x.columns), #예측 변수명
               class_names = ['high', 'low'], #타겟 클래스 알파벳순
               proportion = True, #비율 표기
               filled = True, #색칠
               rounded = True, #둥근 테두리
               impurity = False, #불순도 표시
               label = 'root', #label 표시 위치
               fontsize = 10);

#모델을 이용해 예측하기
test_x = df_test.drop(columns = 'income') #예측 변수 추출
test_y = df_test['income'] #타겟 변수 추출
#예측값 구하기
df_test['pred'] = model.predict(test_x)
df_test

#성능 평가
#confusion matrix 만들기
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_true = df_test['income'], #실제값
                            y_pred = df_test['pred'], #예측값
                            labels = ['high', 'low']) #클래스 배치 순서
conf_mat

#히트맵
plt.rcParams.update(plt.rcParamsDefault) #그래프 설정 되돌리기

from sklearn.metrics import ConfusionMatrixDisplay
p = ConfusionMatrixDisplay(confusion_matrix = conf_mat,
                           display_labels = ('high', 'low'))
p.plot(cmap = 'Blues') #컬러맵 적용->출력

import sklearn.metrics as metrics

#accuracy(정확도) - 필수
metrics.accuracy_score(y_true = df_test['income'], #실제값
                       y_pred = df_test['pred'],) #예측값

#precision(정밀도) - 관심 클래스 분명
metrics.precision_score(y_true = df_test['income'], #실제값
                        y_pred = df_test['pred'], #예측값
                        pos_label = 'high') #관심 클래스

#recall(재현율) - 관심 클래스 max
metrics.recall_score(y_true = df_test['income'],
                     y_pred = df_test['pred'],
                     pos_label = 'high')

#f1 score(recall * precision)
metrics.f1_score(y_true = df_test['income'],
                 y_pred = df_test['pred'],
                 pos_label = 'high')

In [ ]:
#p383 조건 충족하는 행 추출
import pandas as pd
df_raw = pd.read_csv('exam.csv')
df = df_raw.head(10)
df

#nclass 추출
df['nclass']

#논리연산자 이용
df['nclass'] == 1 #True/False 출력
df[df['nclass'] == 1] #nclass 1인 것만 추출
df[df['math'] >= 80] #math 80이상 추출

df[(df['nclass'] == 1) & (df['math'] >= 50)] #and
df[(df['math'] < 50) | (df['english'] < 50)] #or

In [ ]:
#p386 열 추출
import pandas as pd
df_raw = pd.read_csv('exam.csv')

df['id'] #series
df[['id']] #dataFrame구조 유지
df[['id', 'nclass']]

df.math #dataFrame 추출 불가
df.math.mean()

In [ ]:
#p388 조건 충족하는 행에서 열 추출
import pandas as pd
df_raw = pd.read_csv('exam.csv')

#nclass 1인 행의 math열
df[df['nclass'] == 1]['math']

#nclass 1인 행 math, english열
df[df['nclass'] == 1][['math', 'english']]

#둘은 같은 코드(pandas 함수와 [] 함께 사용 가능)
df.groupby('nclass')\
  .agg(math = ('math', 'mean'),
       english = ('english', 'mean'))

df.groupby('nclass')[['math', 'english']].mean()

In [ ]:
#p390 df.loc[] 이용해 인덱스 활용
df = pd.DataFrame({'var1': [1, 2, 3],
                   'var2': [4, 5, 6]},
                   index = ['kim', 'lee', 'park']) #인덱스 문자열 지정

#kim행 추출
df.loc['kim']
#kim, park행 추출
df.loc[['kim', 'park']]

#lee행 var1열 추출
df.loc['lee', 'var1']
#kim, lee행 var2열 추출
df.loc[['kim', 'lee'], 'var2']

#모든 행 추출
df.loc[:, 'var1']

In [33]:
#p392 df.loc[] 이용
import pandas as pd
df_raw = pd.read_csv('exam.csv')

#index 번호 있는 df
df = df_raw.copy()
df

#index 번호 행 추출
df.loc[0] #series
df.loc[[2]] #dataFrame
df.loc[[0, 3, 5]] #여러 행 추출

df.loc[0:3] #연속 행 추출(0-3) //slicing
df.loc[:2] #(처음-2)
df.loc[5:] #(5-마지막)
df.loc[:] #(처음-마지막) = df

df.loc[2, :]
#df.loc[:, 2] //error (index번호로 열 추출 불가)

#인덱스 문자열 있는 데이터 프레임 (번호 사용시 error)
df_label = pd.DataFrame({'var1': [1, 2, 3],
                         'var2': [4, 5, 6]},
                         index = ['kim', 'lee', 'park'])

#df_label.loc[0] //error

#p397 조건 충족하는 행 추출
df.loc[df['nclass'] == 1]
df.loc[df['math'] >= 60]

#조건 충족하는 행에서 열 추출
df.loc[df['nclass'] == 1, ['math', 'english']] #조건, 변수명
df.loc[0:3, 'math'] #0-3행 math열

#math > 50 행 추출
df.query('math > 50')
df[df['math'] > 50]
df.loc[df['math'] > 50]

In [ ]:
#p399 df.iloc[] 이용
import pandas as pd
df_raw = pd.read_csv('exam.csv')
df = df_raw.copy()

#index 번호 지정해 행 추출
df.iloc[0] #series
df.iloc[[0]] #dataFrame
df.iloc[[0, 3, 5]] #여러 행 추출

df.iloc[1:5] #연속 행 추출(1-4)

#인덱스 문자열 있는 데이터 프레임 (번호 사용 가능)
df_label = pd.DataFrame({'var1': [1, 2, 3],
                         'var2': [4, 5, 6]},
                         index = ['kim', 'lee', 'park'])

df_label.iloc[0]

#index 번호 지정해 열 추출
df.iloc[:, 1]
df.iloc[:, [1, 3]] #모든 행 1, 3열 추출

df.iloc[2, 3] #2행 3열 추출
df.iloc[[0, 1], [2, 3]] #0, 1행 2, 3열 추출

#조건 충족하는 행 추출 (불가)
#df.iloc[df['nclass] == 1] //error

In [ ]:
#p407 자료 구조
import pandas as pd
df = pd.DataFrame([1, 2, 3]) 
df.mean()

x = [1, 2, 3] #list
#x.mean() //error
sum(x) / len(x)

#df로 자료구조 변경
df_x = pd.DataFrame(x)
df_x.mean()

In [ ]:
#스칼라
x = 1 #int
x = 1.5 #float
x = 'apple' #str

#타입 바꾸기
var1 = 1 #int
var2 = float(var1) #float
var3 = str(var2) #str

In [ ]:
#리스트
x = [1, 2, 'a', 'b']
type(x) #list

x[0] #0번째 값 추출
x[1:3] #1-2번째 값 추출

x.append(8) #8 추가
x.remove(2) #2 삭제
x[0] = 9 #0번째 9로 변경
x

In [ ]:
#튜플 (값 수정 불가)
x = (1, 2, 'a', 'b')
type(x) #tuple

x2 = 1, 2, 'a', 'b'
x2 #() 생략 가능

x[0] #0번째 값 추출
x[1:3] #1-2번째 값 추출

#x.append(8) //error
#x.remove(3) //error
#x[0] = 9 //error

In [ ]:
#딕셔너리
x = {'name': 'kim'}
type(x) #dict

#값에 스칼라, 리스트, 튜플 등 다양한 자료구조 이용 가능
x = {'id': 1, 'name': ['kim', 'park'], 'nclass': (1, 2, 3)}
x

#key 이용해 추출
x['id']
x['name']
#x[1] //error (index 사용 불가)

#key에 들어있는 값 개별 요소 인덱스 사용 추출
x['name'][0]
x['nclass'][1]

#딕셔너리에 키-값 쌍 추가
x['score'] = [5, 6, 7]
del(x['id']) #키-값 쌍 삭제
del(x['name'][1]) #name키 1번째 요소 삭제

#값 변경(새 값 할당)
x['score'] = [7, 8, 9]
x['score'][2] = 99 #score 2번째 값 변경
x['math'] = x['score'] #기존key 새 key에 할당
del(x['score']) #기존key 삭제
x

In [ ]:
#시리즈
import pandas as pd
x_num = pd.Series([3, 4, 5])
type(x_num) #series

#index 지정
x_label = pd.Series([3, 4, 5], index = ['kim', 'lee', 'park'])
x_label

x_num[0] #0번째 값 추출
x_label['kim'] #index kim 값 추출
x_label.loc['kim']
x_num.iloc[0]

#새로 만들 인덱스에 값 할당
x_num[3] = 9 
x_label['choi'] = 6

#인덱스 지정해 삭제
del(x_num[2])
del(x_label['park'])

#값 변경
x_num[3] = 4
x_label['kim'] = 5

In [ ]:
#데이터 프레임
import pandas as pd
df = pd.DataFrame({'var1': [1, 2, 3],
                   'var2': [4, 5, 6]})
type(df) #DataFrame

#인덱스 정하기
df = pd.DataFrame({'var1': [1, 2, 3],
                   'var2': [4, 5, 6]},
                   index = ['kim', 'lee', 'park'])
df

x = df['var1']
type(x) #series

#자료 구조 바꾸기
x = [1, 1, 2, 3]
#x.mean() //error(list가 아닌 DataFrame)
#x.value_counts() //error

df_x = pd.DataFrame(x)
type(df_x) #DataFrame
df_x.mean()
df_x.value_counts()